# 2. Data Cleaning

Before feeding the dataset to the dimensionality reduction and clustering algorithms, it must be cleaned.  
This notebook contains:

1. Cleaning steps
2. Assumptions about the data
3. Modeling decisions
4. First approach to feature selection by dealing with missing values

For modeling refer to the notebook: __3_OSMI_EDA.ipynb__ 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', None)

import os
from pathlib import Path
from dotenv import load_dotenv

from sklearn.impute import SimpleImputer

In [ ]:
load_dotenv(Path().cwd().parent.joinpath(".env"))

In [ ]:
# Dataset location
LOCATION_DATSET = Path(os.getenv("LOCATION_DATASET"))

In [ ]:
df = pd.read_csv(LOCATION_DATSET)
n_participants, n_questions = df.shape
print(f"Survey is composed of\nParticipants = {n_participants}\nQuestions = {n_questions}")

In [ ]:
df.head()

# Data Cleaning

## 1. How many people did not answer the majority of the questions

In [ ]:
# 20 people did not answer 50% or more of the survey
# These pariticpants can be discarded
print(((df.isna().sum(axis=1) / n_questions) >= 0.5).sum())

In [ ]:
df = df.loc[(df.isna().sum(axis=1) / n_questions) <= .5,:]
df

## 2. Dealing with the most unanswered questions.

In [ ]:
# what is the proportion of missing values we are dealing with? --> See 1. OSMI_overview
missing_values = df.isna().sum() / len(df)

In [ ]:
# All features that have more than 50% of missing values will be discarded
questions_to_drop = missing_values[missing_values > 0.5].index.tolist()
questions_to_drop

In [ ]:
df = df.drop(columns=questions_to_drop)
df.shape

In [ ]:
# possible categories per question
# How many unique answers? This will help find the questions with non-standarized questions
df.nunique()

In [ ]:
# These two questions are open, so the answers are always very different, 80% of them are answered so they might contain interesting information.
# We extract them into different variables to do a separate analysis later - TODO: sentiment analysis? Bag of Word to check which word are most frequent
# indexes are kept to be able to identify participants later in the clusters
# main question: Would you be willing to bring up a physical health issue with a potential employer in an interview?
# secondary:
DATA_UTILS = Path(os.getenv("DATA_UTILS"))
willingness_to_bring_up_physical_issue = df.loc[:,"Why or why not?"]
willingness_to_bring_up_physical_issue.to_csv(DATA_UTILS.joinpath("willingness_to_bring_up_physical_issue.csv"), sep=";")

In [ ]:
# main questions: Would you bring up a mental health issue with a potential employer in an interview?
willingness_to_bring_up_mentall_issue = df.loc[:,"Why or why not?.1"]
willingness_to_bring_up_mentall_issue.to_csv(DATA_UTILS.joinpath("willingness_to_bring_up_mental_issue.csv"), sep=";")

In [ ]:
# Sanity check
df = df.drop(columns=["Why or why not?", "Why or why not?.1"])
df.nunique()

In [ ]:
n_participants, n_questions = df.shape
print(f"Survey is composed of\nParticipants = {n_participants}\nQuestions = {n_questions}")

## 2. Cleaning specific questions
### Gender
To facilitate the analysis, all answere were classified in three different groups: male, female, non-binary.  
This solely with the purpose facilitate the analysis it does not intent to under represent any specific gender.

In [ ]:
# Dealing with specific questions
gender = df.loc[:, "What is your gender?"].copy()
gender = gender.str.lower()
gender = gender.str.strip()
gender_list = gender.unique().tolist()
gender_list

In [ ]:
non_binary = [
    'bigender',
    'non-binary',
    'transitioned, m2f',
    'genderfluid (born female)',
    'other/transfeminine',
    'androgynous',
    'male 9:1 female, roughly',
    'other',
    'nb masculine',
    'none of your business',
    'genderqueer',
    'human',
    'genderfluid',
    'enby',
    'genderqueer woman',
    'mtf',
    'queer',
    'agender',
    'fluid',
    'male/genderqueer',
    'fem',
    'nonbinary',
    'unicorn',
    'male (trans, ftm)',
    'genderflux demi-girl',
    'transgender woman',
    'afab',
    'female assigned at birth',
    'female-bodied; no feelings about gender'
]

In [ ]:
gender = gender.apply(lambda x : "non-binary" if x in non_binary else x)
gender

In [ ]:
gender.unique().tolist()

In [ ]:
binary = {
    'm' : 'male',
    'i identify as female.' : 'female',
    'f' : 'female',
    'woman' : 'female',
    'man' : 'male',
    'fm' : 'female',
    'cis female' : 'female',
    'female or multi-gender femme' : 'female',
    'female/woman' : 'female',
    'cis male' : 'male',
    'male.' : 'male',
    'male (cis)' : 'male',
    'cisgender female' : 'female',
    'sex is male' : 'male',
    'malr' : 'male',
    'dude' : 'male',
    "i'm a man why didn't you make this a drop down question. you should of asked sex? and i would of answered yes please. seriously how much text can this take?" : 'male',
    'mail' : 'male',
    'm|' : 'male',
    'female (props for making this a freeform field, though)' : 'female',
    'cis-woman' : 'female',
    'cisdude' : 'male',
    'cis man' : 'male'
}

In [ ]:
gender = gender.replace(binary)
gender

In [ ]:
gender.unique()

In [ ]:
df["What is your gender?"] = gender

In [ ]:
# Sanity check
# Compare unique answers with nans and without nans. What other questions requieres especial attention?
unique_vals_with_nan = df.nunique(dropna=False)
unique_vals_with_nan.name = "with_nan"

unique_vals_without_nan = df.nunique(dropna=True)
unique_vals_without_nan.name = "without_nan"

pd.merge(unique_vals_with_nan, unique_vals_without_nan, left_index=True, right_index=True)

### Country of residence

In [ ]:
# Exchange country names by the ISO 3166 encoding to have a unified representation and avoid special characters in country names
countries = df["What country do you live in?"].value_counts(dropna=False)
countries

In [ ]:
# replace country names with their iso code
COUNTRY_CODES = Path(os.getenv("COUNTRY_CODES"))
country_codes = pd.read_csv(COUNTRY_CODES, sep=";", encoding="utf-8").set_index("country")
country_codes.head()

In [ ]:
df["What country do you live in?"] = df["What country do you live in?"].map(lambda x : country_codes.loc[x].values[0] if x in country_codes.index else x)
df["What country do you live in?"].value_counts()

### Country of work

In [ ]:
df["What country do you work in?"] = df["What country do you work in?"].map(lambda x : country_codes.loc[x].values[0] if x in country_codes.index else x)
df["What country do you work in?"].value_counts()

In [ ]:
df.loc[:, ["What country do you live in?", "What country do you work in?"]].to_csv(DATA_UTILS.joinpath("countries.csv"))

In [ ]:
df = df.drop(columns=["What country do you live in?", "What country do you work in?"])

### US State codes

In [ ]:
US_STATE_CODES = Path(os.getenv("US_STATE_CODES"))
us_state_codes = pd.read_csv(US_STATE_CODES, sep=";", encoding="utf-8").set_index("state")
us_state_codes

In [ ]:
df["What US state or territory do you live in?"] = df["What US state or territory do you live in?"].map(lambda x : us_state_codes.loc[x].values[0] if x in us_state_codes.index else x)
df["What US state or territory do you work in?"] = df["What US state or territory do you work in?"].map(lambda x : us_state_codes.loc[x].values[0] if x in us_state_codes.index else x)
df

In [ ]:
df.loc[:, ["What US state or territory do you live in?", "What US state or territory do you work in?"]].to_csv(DATA_UTILS.joinpath("us_states.csv"))

In [ ]:
df = df.drop(columns=["What US state or territory do you live in?", "What US state or territory do you work in?"])

In [ ]:
# Sanity check
# How many unique answers with and without nans? Are theere any questions that still requiere special attention?
unique_vals_with_nan = df.nunique(dropna=False)
unique_vals_with_nan.name = "with_nan"

unique_vals_without_nan = df.nunique(dropna=True)
unique_vals_without_nan.name = "without_nan"

pd.merge(unique_vals_with_nan, unique_vals_without_nan, left_index=True, right_index=True)

### Cleaning work position

In [ ]:
df["Which of the following best describes your work position?"].value_counts(dropna=False)

In [ ]:
# separate the roles and keep only the first one
# We assume people with many responsibilities tend to communicate everything they are responsible for but mention the main task responsibility first
# This feature is considered important since it can help identify clusters, soo we keep the first role mentioned by the participants
roles = df["Which of the following best describes your work position?"].str.split("|", expand=True)
roles = roles.apply(lambda col: col.str.strip())
roles.columns = [f"role_{n}" for n in range(10)]
roles

In [ ]:
# Sanity check - answers seem standardized we can keep them as is
roles["role_0"].value_counts()

In [ ]:
df["Which of the following best describes your work position?"] = roles["role_0"]
df

### Diagnosed disorders

Similarly to the "Why?" questions the diagnosed disorders seem to be free text and there is quite some overlapping in the responses.  
Separte the disorders and keep only the first one as we did with the roles does not seem feasible.

Variable will be extracted for separate evaluation as it was with the why questions.

In [ ]:
# What diagnosis are observed?
diagnosis = df["If so, what condition(s) were you diagnosed with?"].str.split("|", expand=True)
diagnosis.columns = [f"diagnosis_{n}" for n in range(len(diagnosis.columns))]
diagnosis

In [ ]:
diagnosis["diagnosis_0"].value_counts()

In [ ]:
diagnosis["diagnosis_1"].value_counts()

In [ ]:
diagnosis["diagnosis_2"].value_counts()

In [ ]:
diagnosis["diagnosis_3"].value_counts()

In [ ]:
df = df.drop(columns="If so, what condition(s) were you diagnosed with?")
df

In [ ]:
n_participants, n_questions = df.shape
print(f"Survey is composed of\nParticipants = {n_participants}\nQuestions = {n_questions}")

In [ ]:
# Sanity check
# How many unique answers with and without nans? Are theere any questions that still requiere special attention?
unique_vals_with_nan = df.nunique(dropna=False)
unique_vals_with_nan.name = "with_nan"

unique_vals_without_nan = df.nunique(dropna=True)
unique_vals_without_nan.name = "without_nan"

pd.merge(unique_vals_with_nan, unique_vals_without_nan, left_index=True, right_index=True)

### Age

In [ ]:
# Overview of the age. We immediately see that the min value is 3 and the may value is 323
# We assume the presence of outliers
df["What is your age?"].describe()

In [ ]:
# Take a look at the unique ages and how many people are in that age
df["What is your age?"].value_counts(dropna=False).sort_index()

In [ ]:
# Explore distribution. Histogram and box plot confirm outliers
fig, axs = plt.subplots(figsize=(16,6), nrows=1, ncols=2)
sns.histplot(data=df, x="What is your age?", stat="density", kde=True, ax=axs[0])
sns.boxplot(data=df, x="What is your age?", ax=axs[1])

In [ ]:
# the age of the outliers will be imputed with the median
# any age that is larger than q75 + 1.5*iqr or q25 - 1.5*iqr will be considered an outlier
q25, q75 = np.quantile(df["What is your age?"], [.25, .75])
iqr = q75 - q25
iqr

In [ ]:
upper_limit = q75 + (1.5 * iqr)
lower_limit = q25 - (1.5 * iqr)

In [ ]:
upper_limit

In [ ]:
lower_limit

In [ ]:
df.loc[:,["What is your age?"]][(df["What is your age?"] > upper_limit) | (df["What is your age?"] < lower_limit)].shape

In [ ]:
# Proportion of imputed ages -> low impact on the dataset
26/1413

In [ ]:
df["What is your age?"] = df["What is your age?"].apply(lambda age: df["What is your age?"].median() if age > upper_limit or age < lower_limit else age)

In [ ]:
# Sanity check
# How many unique answers with and without nans? Are theere any questions that still requiere special attention?
unique_vals_with_nan = df.nunique(dropna=False)
unique_vals_with_nan.name = "with_nan"

unique_vals_without_nan = df.nunique(dropna=True)
unique_vals_without_nan.name = "without_nan"

pd.merge(unique_vals_with_nan, unique_vals_without_nan, left_index=True, right_index=True)

### Overview of the whole dataset.

Is everything clean? Are there any more questions that requiere special attention and standardization?

In [ ]:
for col in df.columns:
    print(df[col].value_counts(dropna=False))
    print()

### Are you self-employed

Missing answers in some of the questions are not random, they are the consequence of the first question.  
For self-employed people some of the question simply do not apply so they were left unanswered, in this case missingness provides structure. This can be a potential feature for the clusters

In [ ]:
# observing a smallsubset of the self employed participants
df[df["Are you self-employed?"] == 1].head(10)

In [ ]:
df.loc[:, 
    ["Do you have previous employers?", "Have you observed or experienced an unsupportive or badly handled response to a mental health issue in your current or previous workplace?"]][df["Have you observed or experienced an unsupportive or badly handled response to a mental health issue in your current or previous workplace?"].isna()]

# Dealing with missing values

During the exploration of the dataset it was detected that the missing values observed ar not Missing At Random, they are the consequence of how the survey has beed structured.  
For example, the question "Are you self employed?" Revealed that 267 people are indeed self-employed which has an impact on following questions about their current work place such as:  

- How many employees does your company or organization have?
- Is your employer primarily a tech company/organization?
- Does your employer provide mental health benefits as part of healthcare coverage?
- etc.

There are 13 questions that are directly affected by this first question.

Similarly, for the question: _Do you have previous employers?_, there are 149 people who do not have previous employers and some of them overlap with the people who are self-employed.  
This indicates some normal career paths where people might have started a bussiness or do freelancing.

Missing values will be encoded with a new category depending on the question.

In [ ]:
# Check how many missing values per question after standardizing answers
df.isna().sum()

In [ ]:
# We use the NA category to replace missing values in the text columns
na_questions = [
    "How many employees does your company or organization have?",
    "Does your employer provide mental health benefits as part of healthcare coverage?",
    "Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?",
    "Does your employer offer resources to learn more about mental health concerns and options for seeking help?",
    "Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?",
    "If a mental health issue prompted you to request a medical leave from work, asking for that leave would be:",
    "Do you think that discussing a mental health disorder with your employer would have negative consequences?",
    "Do you think that discussing a physical health issue with your employer would have negative consequences?",
    "Would you feel comfortable discussing a mental health disorder with your coworkers?",
    "Would you feel comfortable discussing a mental health disorder with your direct supervisor(s)?",
    "Do you feel that your employer takes mental health as seriously as physical health?",
    "Have you heard of or observed negative consequences for co-workers who have been open about mental health issues in your workplace?",
    "Have your previous employers provided mental health benefits?",
    "Were you aware of the options for mental health care provided by your previous employers?",
    "Did your previous employers ever formally discuss mental health (as part of a wellness campaign or other official communication)?",
    "Did your previous employers provide resources to learn more about mental health issues and how to seek help?",
    "Was your anonymity protected if you chose to take advantage of mental health or substance abuse treatment resources with previous employers?",
    "Do you think that discussing a mental health disorder with previous employers would have negative consequences?",
    "Do you think that discussing a physical health issue with previous employers would have negative consequences?",
    "Would you have been willing to discuss a mental health issue with your previous co-workers?",
    "Would you have been willing to discuss a mental health issue with your direct supervisor(s)?",
    "Did you feel that your previous employers took mental health as seriously as physical health?",
    "Did you hear of or observe negative consequences for co-workers with mental health issues in your previous workplaces?"
]

# We use the category 2 to encode NA for the columns that are already numeric
na_questions_numeric = [
    "Is your employer primarily a tech company/organization?",
    "Do you have previous employers?",
]

# It is difficult to associate this question with the NA -Not applicable- category since there are 400 NaN,i.e., unanswered questions
# Use the Unknown category to replace the missing values
# It is important to keep it, 71% of people answered it
unknown_questions = [
    "Do you know the options for mental health care available under your employer-provided coverage?",
    "Have you observed or experienced an unsupportive or badly handled response to a mental health issue in your current or previous workplace?"
]

In [ ]:
# imputation
text_imputer = SimpleImputer(strategy="constant", fill_value="Not applicable")
df.loc[:, na_questions] = text_imputer.fit_transform(df.loc[:, na_questions].copy())

numeric_imputer = SimpleImputer(strategy="constant", fill_value=2)
df.loc[:, na_questions_numeric] = numeric_imputer.fit_transform(df.loc[:, na_questions_numeric].copy())

unknown_imputer = SimpleImputer(strategy="constant", fill_value="unknown")
df.loc[:, unknown_questions] = unknown_imputer.fit_transform(df.loc[:, unknown_questions].copy())

gender_imputer = SimpleImputer(strategy="most_frequent")
df.loc[:, "What is your gender?"] = gender_imputer.fit_transform(df.loc[:, ["What is your gender?"]])

In [ ]:
df.nunique()

In [ ]:
df.isna().sum()

In [ ]:
df.to_csv(LOCATION_DATSET.parent.joinpath("osmi_mental_health_clean.csv"))

In [ ]:
df